# Extracting Data from Spotify API

In [ ]:
#import spotipy
#from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
# Setting environement variables
# https://www.youtube.com/watch?v=YdgIWTYQ69A

#from dotenv import load_dotenv
#import os

#load_dotenv()

#cid = os.getenv("cid")
#secret = os.getenv("secret")

In [ ]:
#Authentication 
#client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
#sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

# Fetching Spotify Tracks
We'll extract 

In [6]:
# Importing modules
import sys
sys.path.append("../spotify_api_web_app")

import authorization
import pandas as pd
from tqdm import tqdm
import time
import random

sp = authorization.authorize()

In [3]:
# Fetching genres
genres = sp.recommendation_genre_seeds()

# We are only able to extract 100 recommendations per genre
n_recs = 75

data_dict = {"id":[], "genre":[], "track_name":[], "artist_name":[],
             "danceability":[], "valence":[], "energy":[], "tempo":[],
             "loudness":[], "speechiness":[], "instrumentalness":[]}


In [5]:
# DATA CRAWL
# Get recs for every genre
for g in tqdm(genres):
    
    # Get n recommendations
    recs = sp.recommendations(genres = [g], limit = n_recs)
    # json-like string to dict
    recs = eval(recs.json().replace("null", "-999").replace("false", "False").replace("true", "True"))["tracks"]
    
    # Crawl data from each track 
    for track in recs:
        # ID and Genre
        data_dict["id"].append(track["id"])
        data_dict["genre"].append(g)
        # Metadata
        track_meta = sp.track(track["id"])
        data_dict["track_name"].append(track_meta.name)
        data_dict["artist_name"].append(track_meta.album.artists[0].name)
        # Mood Features
        track_features = sp.track_audio_features(track["id"])
        data_dict["valence"].append(track_features.valence)
        data_dict["energy"].append(track_features.energy)
        data_dict["danceability"].append(track_features.danceability)
        data_dict["tempo"].append(track_features.tempo)
        #Properties Features
        data_dict["loudness"].append(track_features.loudness)
        data_dict["speechiness"].append(track_features.speechiness)
        data_dict["instrumentalness"].append(track_features.instrumentalness)

        # Wait 0.2 seconds per track so that the api doesn't overheat
        time.sleep(0.2)


100%|██████████| 126/126 [1:09:06<00:00, 32.90s/it]


In [13]:
# Processing Data
#df = pd.DataFrame(data_dict)

df = pd.DataFrame.from_dict(data_dict, orient='index')
df = df.transpose()

In [14]:
# CONVERT TO CSV ONCE THIS RUNS AGAIN!!
df.to_csv("track_properties_dataset.csv", index = False)
df = pd.read_csv("track_properties_dataset.csv")
df.head()

,id,genre,track_name,artist_name,danceability,valence,energy,tempo,loudness,speechiness,instrumentalness
0,6wN5nWLgjEGmPwozDh2jwM,acoustic,Kiss Me,Jason Walker,0.429,0.330,0.184,77.567,-9.731,0.0299,0.000000
1,5z1vCywwTJAP8qSkGJymFY,acoustic,Creep - Live,Korn,0.484,0.236,0.534,104.023,-8.156,0.1510,0.000000
2,1ZmbLKxfrWh2ejFjiZxZPW,acoustic,Better Off - Live From Spotify London,HAIM,0.510,0.419,0.702,119.296,-10.475,0.0296,0.003800
3,7noCkklUhEAoj8GJkbAuHq,acoustic,All of Me,Boyce Avenue,0.606,0.308,0.209,125.969,-10.273,0.0418,0.000000
4,6wDKGtbPpF9xPMRv7bswQN,acoustic,Dead Sea,The Lumineers,0.713,0.472,0.319,94.043,-11.077,0.0289,0.000079
